In [0]:
# # install pytorch
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision


In [0]:
# # Install pillow specific for collab other verisions cause issue
# !pip install pillow==4.1.1
# %reload_ext autoreload
# %autoreload

In [2]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [29]:
torch.__version__

'1.1.0'

In [0]:
# Go to current working directory
import os
os.chdir('drive/My Drive/HandGestures/')

In [4]:
ls

 dataset.py           nus.csv                         __pycache__/
 handgestures.ipynb  'NUS Hand Posture dataset-II'/


In [0]:
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms, models
import os,dataset,copy,pickle

In [0]:
# Download dataset
# !wget https://www.ece.nus.edu.sg/stfpage/elepv/NUS-HandSet/NUS-Hand-Posture-Dataset-II.zip

In [0]:
# Unzipping file
# !unzip NUS-Hand-Posture-Dataset-II.zip

In [0]:
dataset_path = './NUS Hand Posture dataset-II'
destination_path = './'

In [0]:
# !rm 'NUS Hand Posture dataset-II/Hand Postures with human noise/Thumbs.db'

In [10]:
df, train_df, test_df = dataset.create_and_load_meta_csv_df(dataset_path, destination_path, split=0.80)

File:- nus
{0: 'd'}
{0: 'd', 1: 'a'}
{0: 'd', 1: 'a', 2: 'i'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c', 4: 'f'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c', 4: 'f', 5: 'g'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c', 4: 'f', 5: 'g', 6: 'h'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c', 4: 'f', 5: 'g', 6: 'h', 7: 'b'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c', 4: 'f', 5: 'g', 6: 'h', 7: 'b', 8: 'e'}
{0: 'd', 1: 'a', 2: 'i', 3: 'c', 4: 'f', 5: 'g', 6: 'h', 7: 'b', 8: 'e', 9: 'j'}


In [11]:
train_df.head()

,path,label
0,/content/drive/My Drive/HandGestures/NUS Hand ...,1
1,/content/drive/My Drive/HandGestures/NUS Hand ...,0
2,/content/drive/My Drive/HandGestures/NUS Hand ...,0
3,/content/drive/My Drive/HandGestures/NUS Hand ...,1
4,/content/drive/My Drive/HandGestures/NUS Hand ...,3


In [0]:
with open('nus_model_dict.pkl', 'rb') as f:
    nus_dict = pickle.load(f)

In [13]:
print(len(nus_dict))

10


In [0]:
batch_size = 16
num_epochs = 30
num_classes = len(nus_dict)
feature_extract = True
model_name = "resnet"

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        print("Invalid model name, exiting...")
        exit()
        
    return model_ft, input_size

In [17]:
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:02<00:00, 19114111.99it/s]


In [18]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
model_ft = model_ft.to(device)

In [21]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(degrees=(-45,45), translate=(0.1,0.4)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': dataset.ImageDataset(train_df, transform=data_transforms['train']), 
    'test': dataset.ImageDataset(test_df, transform=data_transforms['test']),
}

dataloaders_dict = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size,shuffle=True,num_workers=8),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size,shuffle=True,num_workers=8)
}

train_len, test_len = len(dataloaders_dict['train']), len(dataloaders_dict['test'])
print("train_len %d test len %d" %(train_len, test_len))

train_len 138 test len 35


In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.cpu().item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print()
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print()
        # deep copy the model
        if phase == 'test' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    print()
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [23]:
for name,param in model_ft.named_parameters():
    print(name)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.we

In [24]:
ct = []
params_to_update = []
for name, param in model_ft.named_parameters():
    if "layer3.1.bn2.bias" in ct:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    ct.append(name)

optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.weight
layer4.0.bn2.weight
layer4.0.bn2.bias
layer4.0.downsample.0.weight
layer4.0.downsample.1.weight
layer4.0.downsample.1.bias
layer4.1.conv1.weight
layer4.1.bn1.weight
layer4.1.bn1.bias
layer4.1.conv2.weight
layer4.1.bn2.weight
layer4.1.bn2.bias
fc.weight
fc.bias


In [0]:
criterion = nn.CrossEntropyLoss()

In [26]:
model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/29
----------

train Loss: 1.9771 Acc: 0.3127


test Loss: 1.0779 Acc: 0.7164

Epoch 1/29
----------

train Loss: 1.1363 Acc: 0.6323


test Loss: 0.6013 Acc: 0.8200

Epoch 2/29
----------

train Loss: 0.8287 Acc: 0.7127


test Loss: 0.3737 Acc: 0.8836

Epoch 3/29
----------

train Loss: 0.6589 Acc: 0.7814


test Loss: 0.3160 Acc: 0.9127

Epoch 4/29
----------

train Loss: 0.5550 Acc: 0.8136


test Loss: 0.2112 Acc: 0.9436

Epoch 5/29
----------

train Loss: 0.5120 Acc: 0.8282


test Loss: 0.1812 Acc: 0.9473

Epoch 6/29
----------

train Loss: 0.4620 Acc: 0.8495


test Loss: 0.2526 Acc: 0.9055

Epoch 7/29
----------

train Loss: 0.4383 Acc: 0.8500


test Loss: 0.1456 Acc: 0.9636

Epoch 8/29
----------

train Loss: 0.4047 Acc: 0.8541


test Loss: 0.1305 Acc: 0.9582

Epoch 9/29
----------

train Loss: 0.3700 Acc: 0.8714


test Loss: 0.1401 Acc: 0.9636

Epoch 10/29
----------

train Loss: 0.3651 Acc: 0.8759


test Loss: 0.1747 Acc: 0.9418

Epoch 11/29
----------

train Loss: 0.3199

In [27]:
path = os.path.join(os.path.abspath(destination_path),'nus_model.pth')
print(path)
torch.save(model_ft,path)

/content/drive/My Drive/HandGestures/nus_model.pth
